In [ ]:
import getpass
user_id = getpass.getuser()

import sys
sys.path.append(f"/home/{user_id}/OTTEHR/")

from ast import literal_eval
from common import *
from mimic_common import *
from multiprocess import Pool
import os
import ot
import ot.plot
import random
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import time
from sklearn.manifold import TSNE
import plotly.express as px
import umap
from matplotlib import pyplot as plt

In [ ]:


output_dir = os.path.join(os.path.expanduser("~"), f"OTTEHR/outputs/mimic")
print(f"Will save outputs to {output_dir}")

""" 
Read in the original dataframe
"""
admid_diagnosis_df = pd.read_csv("../../outputs/mimic/ADMID_DIAGNOSIS.csv", index_col=0, header=0, converters={'ICD codes': literal_eval})
print(admid_diagnosis_df)


"""
Train deep patient model and generate representations for targets and sources
"""

def custom_train_reps(source_features, target_features, n_components, pca_explain=False):
    """ 
    Customized training algorithm for generating target representations and source representations

    :param bool pca_explain: print the explained variance of each components
    
    :returns: target representations, source representations
    """
    source_pca = PCA(n_components=n_components)
    source_reps = source_pca.fit_transform(source_features)

    target_pca = PCA(n_components=n_components)
    target_reps = target_pca.fit_transform(target_features)

    if pca_explain:
        source_exp_var = source_pca.explained_variance_ratio_
        source_cum_sum_var = np.cumsum(source_exp_var)
        target_exp_var = target_pca.explained_variance_ratio_
        target_cum_sum_var = np.cumsum(target_exp_var)
        print("Cummulative variance explained by the source PCA is:", source_cum_sum_var[-1])
        print("Cummulative variance explained by the target PCA is:", target_cum_sum_var[-1])

    return source_reps, target_reps


""" 
Run multiple iterations using linear regression
"""
n_components = 50

# suffix = None
# group_name = 'gender'
# group_1 = 'M'
# group_2 = 'F'

# suffix = None
# group_name = 'ethnicity'
# group_1 =   'HISPANIC_OR_LATINO'
# group_2 = 'ASIAN'
# # group_2 = 'WHITE'

# suffix = None
# group_name = 'marital_status'
# group_1 =   'MARRIED'
# group_2 = 'SEPARATED'

# suffix = None
group_name = 'marital_status'
group_1 =  'MARRIED'
group_2 = 'SEPARATED'

# suffix = None
# group_name = 'insurance'
# group_1 =  'Medicare'
# group_2 = 'Medicaid'

# Select a subset of the data (newborn)
# admid_diagnosis_df = admid_diagnosis_df[admid_diagnosis_df['adm_type'] == 'NEWBORN']
# suffix = "newborn"

# suffix = None
# group_name = 'insurance'

# group_name = 'marital_status'
# groups = ['MARRIED', 'SINGLE', 'WIDOWED', 'DIVORCED', 'SEPARATED']


group_1_count = 120
group_2_count = 100


selected_df = select_df_cts(admid_diagnosis_df, group_name, group_1, group_2, source_count=group_1_count, target_count=group_2_count)

source_features, source_labels, target_features, target_labels = gen_features_duration(selected_df, group_name, group_1, group_2)

source_reps, target_reps = custom_train_reps(source_features, target_features, n_components, pca_explain=False)


In [ ]:
tsne = TSNE(n_components=2, random_state=0)
source_proj = tsne.fit_transform(source_reps)
target_proj = tsne.fit_transform(target_reps)
fig = plt.figure(figsize=(4, 3))
plt.scatter(source_proj[:, 0], source_proj[:, 1], color='red', s=4, label=group_1)
plt.scatter(target_proj[:, 0], target_proj[:, 1], color='blue', s=4, label=group_2)
plt.legend()
# plt.savefig(os.path.join(output_dir, f'{group_name}_{group_1}_vs_{group_2}_embedding.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
group_name = 'marital_status'
groups = ['SEPARATED', 'DIVORCED', 'MARRIED','WIDOWED', 'SINGLE']

# group_name = 'insurance'
# groups = ['Self_Pay', 'Private', 'Government', 'Medicare', 'Medicaid']

In [ ]:
group_1_count = 120
group_2_count = 100

for i in range(len(groups)):
    for j in range(i+1, len(groups)):
        group_1 = groups[i]
        group_2 = groups[j]

        selected_df = select_df_cts(admid_diagnosis_df, group_name, group_1, group_2, source_count=group_1_count, target_count=group_2_count)

        source_features, source_labels, target_features, target_labels = gen_features_duration(selected_df, group_name, group_1, group_2)

        source_reps, target_reps = custom_train_reps(source_features, target_features, n_components, pca_explain=False)

        tsne = TSNE(n_components=2, random_state=0)
        source_proj = tsne.fit_transform(source_reps)
        target_proj = tsne.fit_transform(target_reps)
        fig = plt.figure(figsize=(4, 3))
        plt.scatter(source_proj[:, 0], source_proj[:, 1], color='red', s=4, label=group_1)
        plt.scatter(target_proj[:, 0], target_proj[:, 1], color='blue', s=4, label=group_2)
        # plt.xlim(-9, 9)
        # plt.ylim(-9, 9)
        plt.legend()
        plt.savefig(os.path.join(output_dir, f'{group_name}_{group_1}_vs_{group_2}_embedding.pdf'), bbox_inches='tight')
        plt.show()